In [27]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import pandas as pd
import numpy as np

In [28]:
# Hyperparameters 
input_size = 32*32*3
num_classes = 10
num_epochs = 5
batch_size = 100
lr = 0.01

In [29]:
# SVHN dataset

# Normalizer
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.SVHN(root='./data', split="train",
                                        download=True, transform=transform)

testset = torchvision.datasets.SVHN(root='./data', split="test",
                                       download=True, transform=transform)

# Data loader
train_loader_svhn = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

test_loader_svhn = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


In [30]:
# Logistic regression model.
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(input_size, num_classes),
    torch.nn.LogSoftmax(dim=1) 
)

In [31]:
# Use NLL since we include softmax as part of model.  
criterion = nn.NLLLoss()  

In [32]:
# Train and test functions.
def train(model, train_loader, optimizer, num_epochs, criterion, input_size, log_interval):
    model.train()
    for epoch in range(num_epochs):
        print('Epoch {}'.format(epoch+1))
        for i, (images, labels) in enumerate(train_loader):
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # Log the loss.
            if i % log_interval == 0:
                print('Current loss: {}'.format(loss))

def test(model, test_loader, criterion):
    model.eval()
    test_acc = 0
    total_data = 0
    loss = 0
    with torch.no_grad():
        for _, (images, labels) in enumerate(test_loader):
            output = model(images)
            pred = output.argmax(dim=1, keepdim=True)
            test_acc += pred.eq(labels.view_as(pred)).sum().item()
            total_data += len(images)
            loss = criterion(output, labels)
            
    print('Loss: {}'.format(loss))
    
    test_acc /= total_data
    print('Test accuracy over {} data points: {}%'.format(total_data, test_acc * 100))
    
    return loss.item()

In [33]:
test_losses = []

# SGD

In [34]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr) 
train(model, train_loader_svhn, optimizer, num_epochs, criterion, input_size, 100)

Epoch 1
Current loss: 2.264683723449707
Current loss: 2.2752857208251953
Current loss: 2.339815139770508
Current loss: 2.8752002716064453
Current loss: 1.7038588523864746
Current loss: 2.2808666229248047


KeyboardInterrupt: 

In [ ]:
test_loss = test(model, test_loader_svhn, criterion)
test_losses.append(test_loss)

# SGD Momentum

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9) 
train(model, train_loader_svhn, optimizer, num_epochs, criterion, input_size, 100)

In [ ]:
test_loss = test(model, test_loader_svhn, criterion)
test_losses.append(test_loss)

# SGD Nesterov

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=True)
train(model, train_loader_svhn, optimizer, num_epochs, criterion, input_size, 100)

In [ ]:
test_loss = test(model, test_loader_svhn, criterion)
test_losses.append(test_loss)

# Adagrad

In [ ]:
optimizer = optim.Adagrad(model.parameters(), lr=lr)
train(model, train_loader_svhn, optimizer, num_epochs, criterion, input_size, 100)

In [ ]:
test_loss = test(model, test_loader_svhn, criterion)
test_losses.append(test_loss)

# RMSProp

In [ ]:
optimizer = optim.RMSprop(model.parameters(), lr=0.001)
train(model, train_loader_svhn, optimizer, num_epochs, criterion, input_size, 100)

In [ ]:
test_loss = test(model, test_loader_svhn, criterion)
test_losses.append(test_loss)

# Adam

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr) 
train(model, train_loader_svhn, optimizer, num_epochs, criterion, input_size, 100)

In [ ]:
test_loss = test(model, test_loader_svhn, criterion)
test_losses.append(test_loss)

In [ ]:
col = ['SGD','Momentum','Nesterov','Adagrad','RMSProp','Adam']
df = pd.DataFrame(data=[test_losses], columns=col)
df

In [ ]:
df.to_csv('logistic_regression_svhn_loss.csv')

# Normalize loss

In [ ]:
test_losses = np.asarray(test_losses)
normalized_test_losses = []

mean = np.mean(test_losses)
minus_mean = test_losses - mean
normalized_test_losses.append((minus_mean)/np.linalg.norm(minus_mean))

print(normalized_test_losses)

In [ ]:
col = ['SGD','Momentum','Nesterov','Adagrad','RMSProp','Adam']
df = pd.DataFrame(data=normalized_test_losses, columns=col, index = ['Logistic regression SVHN'])
df

In [ ]:
df.to_csv('logistic_regression_svhn_normalized_loss.csv')